In [1]:
import wicked as w
import numpy as np
import itertools

## N atom cc-pVDZ basis

In [2]:
gamma1 = np.load('save_gamma1.npz')
eta1 = np.load('save_eta1.npz')
lambda2 = np.load('save_lambda2.npz')
lambda3 = np.load('save_lambda3.npz')
print(gamma1)
print(eta1)
print(lambda2)
print(lambda3)

NpzFile 'save_gamma1.npz' with keys: aa, AA
NpzFile 'save_eta1.npz' with keys: aa, AA
NpzFile 'save_lambda2.npz' with keys: aaaa, aAaA, AAAA
NpzFile 'save_lambda3.npz' with keys: aaaaaa, aaAaaA, aAAaAA, AAAAAA


In [3]:
Hbar = np.load('save_Hbar.npz')
Hbar = dict(Hbar)
nocc = Hbar['cc'].shape[0]
nact = Hbar['aa'].shape[0]
nvir = Hbar['vv'].shape[0]

In [4]:
w.reset_space()
# alpha
w.add_space('c', 'fermion', 'occupied', list('klmn'))
w.add_space('v', 'fermion', 'unoccupied', list('efgh'))
w.add_space('a', 'fermion', 'general', list('obqrstuvwxyz'))
#Beta
w.add_space('C', 'fermion', 'occupied', list('KLMN'))
w.add_space('V', 'fermion', 'unoccupied', list('EFGH'))
w.add_space('A', 'fermion', 'general', list('OBQRSTUVWXYZ'))

wt = w.WickTheorem()

s = ['c', 'a', 'C', 'A']

Hops = []
for i in itertools.product(['v+', 'a+', 'c+'],['v', 'a', 'c']):
    Hops.append(' '.join(i))
for i in itertools.product(['V+', 'A+', 'C+'],['V', 'A', 'C']):
    Hops.append(' '.join(i))
for i in itertools.product(['v+', 'a+', 'c+'],['v+', 'a+', 'c+'], ['v', 'a', 'c'], ['v', 'a', 'c']):
    Hops.append(' '.join(i))
for i in itertools.product(['V+', 'A+', 'C+'],['V+', 'A+', 'C+'], ['V', 'A', 'C'], ['V', 'A', 'C']):
    Hops.append(' '.join(i))
for i in itertools.product(['v+', 'a+', 'c+'],['V+', 'A+', 'C+'], ['v', 'a', 'c'], ['V', 'A', 'C']):
    Hops.append(' '.join(i))
Hbar_op = w.op("Hbar", Hops, unique=True)

THT = w.op('bra',s).adjoint() @ Hbar_op @ w.op('c',s)
expr = wt.contract(THT, 0, 0)
mbeq = expr.to_manybody_equation('sigma')

In [6]:
func = w.generate_sigma_build(mbeq, 'Hbar')
print(func)

def build_sigma_vector_Hbar(Hbar, gamma1, eta1, lambda2, lambda3, c):
    sigma = {}
    for key in c.keys():
        sigma[key] = np.zeros(c[key].shape)
    sigma["c"] += -1.00000000 * np.einsum('kl,pl->pk', Hbar["cc"],c["c"],optimize='optimal')
    sigma["a"] += -1.00000000 * np.einsum('ok,pk,bo->pb', Hbar["ac"],c["c"],gamma1["aa"],optimize='optimal')
    sigma["a"] += +0.50000000 * np.einsum('obkq,pk,qrob->pr', Hbar["aaca"],c["c"],lambda2["aaaa"],optimize='optimal')
    sigma["c"] += -1.00000000 * np.einsum('ko,pb,ob->pk', Hbar["ca"],c["a"],gamma1["aa"],optimize='optimal')
    sigma["a"] += -1.00000000 * np.einsum('ob,pr,br,qo->pq', Hbar["aa"],c["a"],gamma1["aa"],gamma1["aa"],optimize='optimal')
    sigma["a"] += +1.00000000 * np.einsum('ob,pr,bqor->pq', Hbar["aa"],c["a"],lambda2["aaaa"],optimize='optimal')
    sigma["c"] += +0.50000000 * np.einsum('kobq,pr,bqor->pk', Hbar["caaa"],c["a"],lambda2["aaaa"],optimize='optimal')
    sigma["a"] += +0.50000000 * np.einsum('obqr,pt,so,qrbt->

### A simple test for the diagonal elements of the active alpha block

In [32]:
c = {}
c['a'] = np.identity(nact)
print(c['a'])

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [43]:
sigma = {}
sigma['a'] = np.zeros((nact, nact))
sigma["a"] += -1.00000000 * np.einsum('ob,pr,br,qo->pq', Hbar["aa"],c["a"],gamma1["aa"],gamma1["aa"],optimize='optimal')
sigma["a"] += +1.00000000 * np.einsum('ob,pr,bqor->pq', Hbar["aa"],c["a"],lambda2["aaaa"],optimize='optimal')
sigma["a"] += +0.50000000 * np.einsum('obqr,pt,so,qrbt->ps', Hbar["aaaa"],c["a"],gamma1["aa"],lambda2["aaaa"],optimize='optimal')
sigma["a"] += -0.50000000 * np.einsum('obqr,pt,rt,qsob->ps', Hbar["aaaa"],c["a"],gamma1["aa"],lambda2["aaaa"],optimize='optimal')
sigma["a"] += +0.25000000 * np.einsum('obqr,pt,qrsobt->ps', Hbar["aaaa"],c["a"],lambda3["aaaaaa"],optimize='optimal')

In [44]:
print(sigma['a'].diagonal())
# The correct value (from the spin-orbital code): [1.00135152 0.37521328 0.37521328 0.37521328]

[1.00135152 0.33720225 0.33720225 0.33720225]


## These terms are missing!!!

In [45]:
sigma["a"] -= +1.00000000 * np.einsum('oBqR,pt,so,qRtB->ps', Hbar["aAaA"],c["a"],gamma1["aa"],lambda2["aAaA"],optimize='optimal')
sigma["a"] += -1.00000000 * np.einsum('bOrQ,pt,rt,sQbO->ps', Hbar["aAaA"],c["a"],gamma1["aa"],lambda2["aAaA"],optimize='optimal')
sigma["a"] += +1.00000000 * np.einsum('OBQR,pt,sQRtOB->ps', Hbar["AAAA"],c["a"],lambda3["aAAaAA"],optimize='optimal')
sigma["a"] += +1.00000000 * np.einsum('oBqR,pt,sqRtoB->ps', Hbar["aAaA"],c["a"],lambda3["aaAaaA"],optimize='optimal')

In [46]:
print(sigma['a'].diagonal())

[1.00135152 0.37521328 0.37521328 0.37521328]
